In [3]:
from langchain_groq import ChatGroq

In [4]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key="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", 
    model_name="llama-3.1-8b-instant"
)
response = llm.invoke("capital of america")
print(response.content)

The capital of the United States of America is Washington, D.C. (short for District of Columbia).


In [11]:
!pip install chromadb

  Obtaining dependency information for protobuf from https://files.pythonhosted.org/packages/79/84/c700d6c3f3be770495b08a1c035e330497a31420e4a39a24c22c02cefc6c/protobuf-4.25.6-cp310-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/413.4 kB ? eta -:--:--
   ------------- -------------------------- 143.4/413.4 kB 4.3 MB/s eta 0:00:01
   ---------------------------------------  409.6/413.4 kB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 413.4/413.4 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.28.2
    Uninstalling protobuf-5.28.2:
      Successfully uninstalled protobuf-5.28.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.67.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.6 which is incompatible.


In [5]:
import chromadb


client = chromadb.PersistentClient(path="./chroma_db")  
collection = client.get_or_create_collection(name="my_collection")


collection.add(
    documents=[
        "This document is about New York",
        "This document is about Delhi"
    ],
    ids=['id1', 'id2']
)

print(" Documents added successfully!")


 Documents added successfully!


In [6]:
all_docs = collection.get()
all_docs

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'metadatas': [None, None],
 'documents': ['This document is about New York',
  'This document is about Delhi'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [7]:
documents = collection.get(ids=["id1"])
documents

{'ids': ['id1'],
 'embeddings': None,
 'metadatas': [None],
 'documents': ['This document is about New York'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [ ]:
results = collection.query(
    query_texts=['Query is about america'],
    n_results=2
)

results

{'ids': [['id1', 'id2']],
 'distances': [[1.2380623011544962, 1.5967676199083023]],
 'metadatas': [[None, None]],
 'embeddings': None,
 'documents': [['This document is about New York',
   'This document is about Delhi']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [19]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://careers.ibm.com/job/20978143/software-developer-intern-2025-san-jose-ca/?codes=WEB_Search_INDIA")
page_data = loader.load().pop().page_content
print(page_data)





















Software Developer Intern: 2025 - US | IBM
















































 



Skip to content



Toggle NavigationIBM CareersEnglish简体中文FrançaisFrançais Canadien日本語PortuguêsEspañolDeutschIBM Careers IBM Careers      English    简体中文    Français    Français Canadien    日本語    Português    Español    Deutsch      
 





Job Details2024-01-04T09:25:13+00:00 

724766BR


Software Developer Intern: 2025    Multiple Locations See All  San Jose  Lowell  Poughkeepsie  Austin    Software Engineering Internship   Apply now Software Developer Intern: 2025    Multiple Locations See All  San Jose  Lowell  Poughkeepsie  Austin    Software Engineering Internship   Apply now IntroductionAt IBM, work is more than a job – it’s a calling: To build. To design. To code. To consult. To think along with clients and sell. To make markets. To invent. To collaborate. Not just to do something better, but to attempt things you’ve never thought possible. Are you ready to lead i

In [20]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [21]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Developer Intern: 2025',
 'experience': 'Summer 2025',
 'skills': ['Programming Languages: Familiarity with one or more programming languages such as Java, Python, C++, or JavaScript.',
  'Development Tools: Experience with software development tools such as Git, IDEs (e.g., Eclipse, IntelliJ), and version control systems.',
  'Problem-Solving Skills: Strong analytical and problem-solving abilities with a keen attention to detail.',
  'Communication: Excellent verbal and written communication skills, with the ability to collaborate effectively in a team environment.',
  'Adaptability: Willingness to learn new technologies and adapt to changing project requirements.',
  'Teamwork: Ability to work both independently and as part of a collaborative team.'],
 'description': 'At IBM, work is more than a job – it’s a calling: To build. To design. To code. To consult. To think along with clients and sell. To make markets. To invent. To collaborate. Not just to do something b

In [17]:
type(json_res)

dict

In [25]:
import pandas as pd
df=pd.read_csv(r"C:\Users\Ajinkya\Downloads\my_portfolio.csv")

In [26]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [22]:
job = json_res
job['skills']

['Programming Languages: Familiarity with one or more programming languages such as Java, Python, C++, or JavaScript.',
 'Development Tools: Experience with software development tools such as Git, IDEs (e.g., Eclipse, IntelliJ), and version control systems.',
 'Problem-Solving Skills: Strong analytical and problem-solving abilities with a keen attention to detail.',
 'Communication: Excellent verbal and written communication skills, with the ability to collaborate effectively in a team environment.',
 'Adaptability: Willingness to learn new technologies and adapt to changing project requirements.',
 'Teamwork: Ability to work both independently and as part of a collaborative team.']

In [29]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}]]

In [30]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Ajinkya, a business development executive at Google. Google is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of google
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Ajinkya, BDE at google
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Your Potential with Google's AI & Software Consulting Expertise

Dear [Hiring Manager's Name],

I came across the job description for a Software Developer Intern at IBM and was impressed by the skills and qualities you're looking for in a candidate. As a Business Development Executive at Google, I'd like to introduce you to our AI & Software Consulting company that can help you find the perfect fit for this role.

At Google, we specialize in facilitating the seamless integration of business processes through automated tools. Our expertise has empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. We'd love to leverage our capabilities to support IBM's goals and objectives.

The skills you've listed in the job description, such as proficiency in programming languages (Java, Python, C++, or JavaScript), experience with software development tools (Git, IDEs, and version control